In [1]:
import numpy as np
import random
import math
import time

def gameBoard(rows=6, columns=7):
    return np.full((rows, columns), ' ', dtype=str)

def legalMoves(board):
    return [col for col in range(board.shape[1]) if board[0, col] == ' ']

def simMoves(board, column, player): 
    temp_board = board.copy()
    for row in range(board.shape[0]-1, -1, -1):
        if temp_board[row, column] == ' ':
            temp_board[row, column] = player
            break
    return temp_board

def check_victory(board, player):
    for r in range(board.shape[0]):
        for c in range(board.shape[1]-3):
            if all(board[r, c+i] == player for i in range(4)):
                return True
    for r in range(board.shape[0]-3):
        for c in range(board.shape[1]):
            if all(board[r+i, c] == player for i in range(4)):
                return True
    for r in range(board.shape[0]-3):
        for c in range(board.shape[1]-3):
            if all(board[r+i, c+i] == player for i in range(4)):
                return True
    for r in range(3, board.shape[0]):
        for c in range(board.shape[1]-3):
            if all(board[r-i, c+i] == player for i in range(4)):
                return True
    return False

def boardstateval(board, player):
    score = 0
    opponent = changePlayer(player)
    rows, cols = board.shape

    for i in range(rows):
        for j in range(cols - 3):

            row_score = sum(board[i, j:j+4] == player)
            score += row_score ** 2  
            row_score_opponent = sum(board[i, j:j+4] == opponent)
            score -= row_score_opponent ** 2

    for i in range(rows - 3):
        for j in range(cols):
            col_score = sum(board[i:i+4, j] == player)
            score += col_score ** 2
            col_score_opponent = sum(board[i:i+4, j] == opponent)
            score -= col_score_opponent ** 2

    return score

def minimax(board, depth, alpha, beta, maximizingPlayer, player):
    legal_moves = legalMoves(board)
    terminal = check_victory(board, 'X') or check_victory(board, 'O') or not legal_moves
    if depth == 0 or terminal:
        if terminal:
            if check_victory(board, player):
                return None, float('inf')  
            elif check_victory(board, 'O' if player == 'X' else 'X'):
                return None, -float('inf') 
            else:
                return None, boardstateval(board, player)  
        else:
            return None, boardstateval(board, player)  

    if maximizingPlayer:
        max_eval = -math.inf
        best_col = random.choice(legal_moves)
        for col in legal_moves:
            row = availableRow(board, col)
            temp_board = simMoves(board, col, player)
            _, eval = minimax(temp_board, depth-1, alpha, beta, False, player)
            if eval > max_eval:
                max_eval = eval
                best_col = col
            alpha = max(alpha, eval)
            if beta <= alpha:
                break
        return best_col, max_eval
    else:
        min_eval = math.inf
        best_col = random.choice(legal_moves)
        for col in legal_moves:
            row = availableRow(board, col)
            temp_board = simMoves(board, col, 'O' if player == 'X' else 'X')
            _, eval = minimax(temp_board, depth-1, alpha, beta, True, player)
            if eval < min_eval:
                min_eval = eval
                best_col = col
            beta = min(beta, eval)
            if beta <= alpha:
                break
        return best_col, min_eval


def availableRow(board, column):
    for r in range(board.shape[0]-1, -1, -1):
        if board[r, column] == ' ':
            return r

def make_move(board, column, player):
    row = availableRow(board, column)
    board[row, column] = player

def changePlayer(player):
    return 'O' if player == 'X' else 'X'

def defaultPlayer(board, player):
    opponent = changePlayer(player)
    legal_moves = legalMoves(board)
    random.shuffle(legal_moves) 

    best_move = None
    for col in legal_moves:
        temp_board = simMoves(board, col, player)
        
        if check_victory(temp_board, opponent):
            return col

       
        if not any(check_victory(simMoves(temp_board, future_col, opponent), opponent) for future_col in legalMoves(temp_board)):
            if check_victory(temp_board, player):
                return col  
            if best_move is None: 
                best_move = col
    
    return best_move if best_move is not None else random.choice(legal_moves)



def play_game(starting_player='X', depth=4):
    board = gameBoard()
    current_player = starting_player
    while True:
        if current_player == 'X':  
            column = defaultPlayer(board, current_player)
        else: 
            column, _ = minimax(board, depth, -math.inf, math.inf, True, current_player)

        make_move(board, column, current_player)
        
        print("Current Board State:")
        print(board)
        
        if check_victory(board, current_player):
            return f"Player {current_player} wins"
        if not legalMoves(board):
            return "Game ends in a draw"
        current_player = changePlayer(current_player)

def simulate_games(num_games, depth, starting_player_type):
    results = {"Player X wins": 0, "Player O wins": 0, "Draws": 0}
    start_time = time.time()
    
    for _ in range(num_games):
        if starting_player_type == "random":
            starting_player = random.choice(['X', 'O'])
        elif starting_player_type == "default_player_X":
            starting_player = 'X'
        elif starting_player_type == "minimax_player_O":
            starting_player = 'O'

        result = play_game(starting_player=starting_player, depth=depth)
        if "wins" in result:
            results[result] += 1
        else:
            results["Draws"] += 1

    elapsed_time = time.time() - start_time
    return results, elapsed_time


In [38]:
results_random, time_taken_random = simulate_games(num_games=100, depth=6, starting_player_type="random")
print("Results (Random starting player):", results_random)
print("Time taken (Random starting player):", time_taken_random, "seconds")

Current Board State:
[[' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' 'X' ' ' ' ' ' ' ' ' ' ']]
Current Board State:
[[' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' 'X' ' ' 'O' ' ' ' ' ' ']]
Current Board State:
[[' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' 'X' ' ' 'O' ' ' 'X' ' ']]
Current Board State:
[[' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' 'O' ' ' ' ' ' ']
 [' ' 'X' ' ' 'O' ' ' 'X' ' ']]
Current Board State:
[[' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' 'O' ' ' '

In [2]:
results_default, time_taken_default = simulate_games(num_games=10, depth=6, starting_player_type="default_player_X")
print("Results (Default player 1 starting player):", results_default)
print("Time taken (Default player 1 starting player):", time_taken_default, "seconds")


Current Board State:
[[' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' 'X' ' ' ' ' ' ' ' ' ' ']]
Current Board State:
[[' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' 'X' ' ' 'O' ' ' ' ' ' ']]
Current Board State:
[[' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ']
 ['X' 'X' ' ' 'O' ' ' ' ' ' ']]
Current Board State:
[[' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ']
 ['X' 'X' ' ' 'O' 'O' ' ' ' ']]
Current Board State:
[[' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' '

: 

In [2]:

results_minimax, time_taken_minimax = simulate_games(num_games=100, depth=6, starting_player_type="minimax_player_O")
print("Results (Minimax player 2 starting player):", results_minimax)
print("Time taken (Minimax player 2 starting player):", time_taken_minimax, "seconds")

Current Board State:
[[' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' 'O' ' ' ' ' ' ']]
Current Board State:
[[' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' 'O' ' ' ' ' 'X']]
Current Board State:
[[' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' 'O' 'O' ' ' ' ' 'X']]
Current Board State:
[[' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' 'X']
 [' ' ' ' 'O' 'O' ' ' ' ' 'X']]


: 